In [2]:
import cv2
import sys
import numpy as np

In [3]:
def findTextRegion(img):
    region = []
 
    # 1. 查找轮廓
    contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
 
    # 2. 筛选那些面积小的
    for i in range(len(contours)):
        cnt = contours[i]
        # 计算该轮廓的面积
        area = cv2.contourArea(cnt) 
 
        # 面积小的都筛选掉
        if(area < 1000):
            continue
 
        # 轮廓近似，作用很小
        epsilon = 0.001 * cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True)
 
        # 找到最小的矩形，该矩形可能有方向
        rect = cv2.minAreaRect(cnt)
        #print "rect is: "
        #print rect
 
        # box是四个点的坐标
        box = cv2.boxPoints(rect)
        box = np.int0(box)
 
        # 计算高和宽
        height = abs(box[0][1] - box[2][1])
        width = abs(box[0][0] - box[2][0])
 
        # 筛选那些太细的矩形，留下扁的
        if(height > width * 1.2):
            continue
 
        region.append(box)
 
    return region

In [4]:
def preprocess(gray):
    # 1. Sobel算子，x方向求梯度
    sobel = cv2.Sobel(gray, cv2.CV_8U, 1, 0, ksize = 3)
    # 2. 二值化
    ret, binary = cv2.threshold(sobel, 0, 255, cv2.THRESH_OTSU+cv2.THRESH_BINARY)
 
    # 3. 膨胀和腐蚀操作的核函数
    element1 = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 9))
    element2 = cv2.getStructuringElement(cv2.MORPH_RECT, (24, 6))
 
    # 4. 膨胀一次，让轮廓突出
    dilation = cv2.dilate(binary, element2, iterations = 1)
 
    # 5. 腐蚀一次，去掉细节，如表格线等。注意这里去掉的是竖直的线
    erosion = cv2.erode(dilation, element1, iterations = 1)
 
    # 6. 再次膨胀，让轮廓明显一些
    dilation2 = cv2.dilate(erosion, element2, iterations = 3)
 
    # 7. 存储中间图片 
    cv2.imwrite("binary.png", binary)
    cv2.imwrite("dilation.png", dilation)
    cv2.imwrite("erosion.png", erosion)
    cv2.imwrite("dilation2.png", dilation2)
 
    return dilation2

In [5]:
def detect(img):
    # 1.  转化成灰度图
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
 
    # 2. 形态学变换的预处理，得到可以查找矩形的图片
    dilation = preprocess(gray)
 
    # 3. 查找和筛选文字区域
    region = findTextRegion(dilation)
 
    # 4. 用绿线画出这些找到的轮廓
    for box in region:
        cv2.drawContours(img, [box], 0, (0, 255, 0), 2)
 
    cv2.namedWindow("img", cv2.WINDOW_NORMAL)
    cv2.imshow("img", img)
 
    # 带轮廓的图片
    cv2.imwrite("contours.png", img)
 
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [6]:
imagePath = '/Users/ryan/OneDrive - The University of Nottingham Ningbo China/Y2/OCR Summer research/Python_Algorithm/OCR Dataset/IMG_0834.JPG'

if __name__ == '__main__':
    # 读取文件
    #imagePath = sys.argv[1]
    img = cv2.imread(imagePath)
    detect(img)

In [8]:
    imagePath = '/Users/ryan/OneDrive - The University of Nottingham Ningbo China/Y2/OCR Summer research/Python_Algorithm/OCR Dataset/IMG_0834.JPG'

    img = cv2.imread(imagePath)   
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
 
    # 2. 形态学变换的预处理，得到可以查找矩形的图片
    dilation = preprocess(gray)
 
    # 3. 查找和筛选文字区域
    region = findTextRegion(dilation)

In [9]:
region[1]

array([[1249, 3499],
       [1249, 3444],
       [1575, 3444],
       [1575, 3499]])

In [18]:
region[2]

array([[ 244, 3442],
       [ 683, 3442],
       [ 683, 3499],
       [ 244, 3499]])

In [11]:
region[3]

array([[1641, 3418],
       [1720, 3418],
       [1720, 3433],
       [1641, 3433]])

In [10]:
type(region)

list

In [14]:
crop_img = img[3444:3499, 1249:1575]

In [16]:
crop_img = img[3444:3499, 1249:1575]
cv2.namedWindow("crop_img", cv2.WINDOW_NORMAL)
cv2.imshow("crop_img", crop_img)
cv2.imwrite("crop_img.png", crop_img)

True

In [26]:
def extractcor(region):

    cropcor = []
    for box in region:

        x = [] ; y = []

        for cordinate in box:
            x.append(cordinate[1])
            y.append(cordinate[0])

        xmax = max(x) ; xmin = min(x)
        ymax = max(y) ; ymin = min(y)

        tempcor = [xmin,xmax,ymin,ymax]
        cropcor.append(tempcor)

    return cropcor

In [27]:
extractcor(region)

[[3444, 3497, 2179, 2903],
 [3444, 3499, 1249, 1575],
 [3442, 3499, 244, 683],
 [3418, 3433, 1641, 1720],
 [3417, 3433, 1306, 1394],
 [3416, 3431, 1763, 1832],
 [3416, 3432, 1544, 1639],
 [3416, 3433, 368, 462],
 [3415, 3432, 2219, 2305],
 [3415, 3435, 1925, 2094],
 [3411, 3434, 2340, 2906],
 [3347, 3410, 472, 1821],
 [3284, 3331, 1511, 1716],
 [3284, 3332, 710, 973],
 [3284, 3331, 518, 704],
 [3283, 3332, 239, 473],
 [3281, 3329, 2007, 2247],
 [3280, 3334, 1058, 1247],
 [3279, 3327, 2419, 2612],
 [3278, 3327, 2681, 2833],
 [3207, 3265, 471, 1818],
 [3146, 3196, 709, 971],
 [3146, 3193, 517, 703],
 [3144, 3194, 1507, 1714],
 [3144, 3193, 240, 471],
 [3141, 3189, 2004, 2244],
 [3141, 3195, 1060, 1246],
 [3139, 3185, 2417, 2608],
 [3137, 3186, 2677, 2830],
 [3070, 3127, 471, 1677],
 [3009, 3054, 709, 971],
 [3008, 3055, 517, 704],
 [3007, 3054, 242, 473],
 [3004, 3052, 1491, 1723],
 [3003, 3056, 1056, 1247],
 [3002, 3049, 2002, 2243],
 [2997, 3044, 2415, 2605],
 [2996, 3043, 2674, 2825],

In [25]:
a = []

SyntaxError: invalid syntax (<ipython-input-25-f208cdf1dc5f>, line 1)

In [22]:
min(a)

1

In [23]:
a = [1,2] ; b = [3,4]

In [24]:
b

[3, 4]

In [ ]:
# filename=0
# filenum=0
# img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# b=20#数字块大小为20×20
# m=img.shape[0]//b
# n=img.shape[1]//b
imagePath = 'IMG_1.jpg'

if __name__ == '__main__':
    # 读取文件
    #imagePath = sys.argv[1]
    img = cv2.imread(imagePath)
    cor=detect(img)

coordinates=cor[::-1]
coordinates_int10=np.floor_divide(coordinates,10)
numberofrows=len(coordinates)

columns=[]
for i in range(numberofrows):
    if i==0 or coordinates_int10[i][0]>coordinates_int10[i-1][0]:
        crop_img = img[coordinates[i][0]:coordinates[i][1], coordinates[i][2]:coordinates[i][3]]
        result = list(pytesseract.image_to_string (crop_img, lang='chi_sim'))
        columns.append(result)

    else:
        if coordinates_int10[i][2]>coordinates_int10[i-1][2]:
            crop_img = img[coordinates[i][0]:coordinates[i][1], coordinates[i][2]:coordinates[i][3]]
            result = list(pytesseract.image_to_string (crop_img, lang='chi_sim'))
            columns[i-1].append(result)

df=pd.DataFrame(columns)
df.fillna('',inplace=True)

# path="result/result.xlsx"
# if not os.path.exists(path):
#     os.makedirs(path)
df.to_excel('result.xlsx',index=False)





#    offsetRow=i*b
#    if i%5==0 and i !=0:
#        filename+=1
#        filenum=0
       
    # for j in range(n):
    #     offsetCol=j*b
    #     filepath=path+"\\"+str(filenum)+".jpg"
    #     filenum+=1
    #     im=img[offsetRow:offsetRow+b-1,offsetCol:offsetCol+b-1]
    #     cv2.imwrite(filepath,im, [int( cv2.IMWRITE_JPEG_QUALITY), 95])
#print(filename,filenum)
#print(path)